In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk
from nltk import word_tokenize
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
import spacy

In [39]:
df = pd.read_csv(r'C:\Users\ritod\Downloads\wb_dat.csv')
df.head()


,Project ID,Region,Country,Project Status,Last Stage Reached Name,Project Name,Project Development Objective,Implementing Agency,Consultant Services Required,Project URL,...,Grant Amount,Borrower,Lending Instrument,Environmental Assessment Category,Environmental and Social Risk,Sector 1,Sector 2,Sector 3,Theme 1,Theme 2
0,P252350,Eastern and Southern Africa,Republic of Rwanda,Active,Bank Approved,Rwanda Priority Skills for Growth (PSG),The Program Development Objective is to expand...,Ministry Of Education,NaN,http://projects.worldbank.org/P252350/null?lan...,...,0.0,Ministry of Finance and Economic Planning,Program-for-Results Financing,NaN,NaN,Public Administration - Education,Tertiary Education,Workforce Development and Vocational Education,Education for the knowledge economy,NaN
1,P252338,East Asia and Pacific,Democratic Republic of Timor-Leste,Active,Bank Approved,Road Climate Resilience Project Second Additio...,The project will deliver sustainable climate r...,"Ministry of Public Works, Transport, and Commu...",NaN,http://projects.worldbank.org/P252338/null?lan...,...,0.0,Ministry of Finance,Investment Project Financing,B,NaN,Public Administration - Transportation,Rural and Inter-Urban Roads,NaN,NaN,NaN
2,P252337,Eastern and Southern Africa,Republic of Kenya,Dropped,Concept Review,Kenya Devolution and Service Delivery,To improve citizen and government access to co...,Kenya National Bureau of Statistics,NaN,http://projects.worldbank.org/P252337/null?lan...,...,340000.0,Kenya National Bureau of Statistics,Investment Project Financing,C,NaN,NaN,NaN,NaN,"Public expenditure, financial management and p...",Decentralization
3,P179254,Western and Central Africa,Republic of Cameroon,Pipeline,NaN,Cameroon Health System Strengthening Project,To increase utilization and quality of reprodu...,NaN,NaN,http://projects.worldbank.org/P179254/null?lan...,...,0.0,NaN,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P179238,Western and Central Africa,Republic of Chad,Pipeline,NaN,Chad Agribusiness and Rural Transformation Pro...,"The project Development Objective (PDO) is ""to...",NaN,NaN,http://projects.worldbank.org/P179238/null?lan...,...,0.0,NaN,Investment Project Financing,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
print(df['Project Status'].unique())

['Active' 'Dropped' 'Pipeline' nan 'Closed']


The attribute Project status has 4 categories: Active, Dropped, Pipeline and Closed alongwith some missing data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21543 entries, 0 to 21542
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Project ID                         21543 non-null  object 
 1   Region                             21541 non-null  object 
 2   Country                            21541 non-null  object 
 3   Project Status                     21539 non-null  object 
 4   Last Stage Reached Name            15196 non-null  object 
 5   Project Name                       21542 non-null  object 
 6   Project Development Objective      11580 non-null  object 
 7   Implementing Agency                6649 non-null   object 
 8   Consultant Services Required       1484 non-null   object 
 9   Project URL                        21542 non-null  object 
 10  Board Approval Date                18786 non-null  object 
 11  Project Closing Date               16584 non-null  obj

In [4]:
df.nunique()

Project ID                           21543
Region                                  11
Country                                207
Project Status                           4
Last Stage Reached Name                  5
Project Name                         18606
Project Development Objective         9862
Implementing Agency                   4621
Consultant Services Required             3
Project URL                          21542
Board Approval Date                   5985
Project Closing Date                  1831
Financing Type                           4
Current Project Cost                  4584
IBRD Commitment                       1301
IDA Commitment                        1237
Total IDA and IBRD Commitment         1912
Grant Amount                           969
Borrower                              2187
Lending Instrument                      18
Environmental Assessment Category        8
Environmental and Social Risk            4
Sector 1                               160
Sector 2   

In [5]:
print(df.isnull().sum())
df.isnull().sum().sum()

Project ID                               0
Region                                   2
Country                                  2
Project Status                           4
Last Stage Reached Name               6347
Project Name                             1
Project Development Objective         9963
Implementing Agency                  14894
Consultant Services Required         20059
Project URL                              1
Board Approval Date                   2757
Project Closing Date                  4959
Financing Type                        8506
Current Project Cost                   148
IBRD Commitment                          1
IDA Commitment                           1
Total IDA and IBRD Commitment            1
Grant Amount                             1
Borrower                             15131
Lending Instrument                     249
Environmental Assessment Category     8461
Environmental and Social Risk        20268
Sector 1                              1916
Sector 2   

158953

In [41]:
for i in range(len(df['Project Status'])):
    if df['Project Status'][i] == 'Closed':
        df['Project Status'][i]= 1
    else:
        df['Project Status'][i]= 0

<ipython-input-41-2cd724344185>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Project Status'][i]= 0
<ipython-input-41-2cd724344185>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Project Status'][i]= 1


Since it has been asked to train a binary classifier to detect whether a project is closed or not, Closed status has been assigned a value of 1 and the other categories have been assigned a value of 0

In [42]:
df.drop(['Project ID','Project URL',
        'Consultant Services Required','Sector 2','Sector 3','Theme 1','Theme 2',
        'Project Name','Implementing Agency','Environmental and Social Risk','Borrower','Board Approval Date','Project Closing Date'], axis=1, inplace=True)

Attributes with lot of missing data have been removed since they do not contribute to the classification

In [43]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Country'] = le.fit_transform(df['Country'])
df['Region'] = le.fit_transform(df['Region'])
df['Last Stage Reached Name'] = le.fit_transform(df['Last Stage Reached Name'])
df['Financing Type'] = le.fit_transform(df['Financing Type'])
df['Environmental Assessment Category'] = le.fit_transform(df['Environmental Assessment Category'])
df['Sector 1'] = le.fit_transform(df['Sector 1'])
df['Project Status'] = le.fit_transform(df['Project Status'])
df['Lending Instrument'] = le.fit_transform(df['Lending Instrument'])

Categotical Data has been encoded with a label encoder

In [44]:
def clean_text(text): 
   
    text = str(text).lower() # convert to lowercase
    text = re.sub('\[.*?\]', '', text) 
    text = re.sub('https?://\S+|www\.\S+', '', text) # Remove URls
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text) # Remove punctuation
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

For the column Project Development Stage, the data has been converted to lowercase,punctions,unecessary symbols have been removed

In [45]:
df['Project Development Objective '] = df['Project Development Objective '].apply(clean_text)

In [46]:
# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['Project Development Objective ']=df['Project Development Objective '].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))


For the column Project Development Stage, the stopwords have been removed and the data has been lemmatized

In [47]:
df['num_characters'] = df['Project Development Objective '].apply(len)
df['num_words'] = df['Project Development Objective '].apply(lambda x:len(nltk.word_tokenize(x)))

Two new columns have been added which shows nunmber of words and number of characters in the column Project Development Stage as a part of feature engineering

In [48]:
df.head()

,Region,Country,Project Status,Last Stage Reached Name,Project Development Objective,Financing Type,Current Project Cost,IBRD Commitment,IDA Commitment,Total IDA and IBRD Commitment,Grant Amount,Lending Instrument,Environmental Assessment Category,Sector 1,num_characters,num_words
0,4,148,0,0,program development objective expand opportuni...,2,120000000.0,0.0,120000000.0,120000000.0,0.0,8,8,127,1025,125
1,3,24,0,0,project deliver sustainable climate resilient ...,4,39140000.0,0.0,35200000.0,35200000.0,0.0,5,1,134,85,9
2,4,125,0,3,improve citizen government access county datum...,4,340000.0,0.0,0.0,0.0,340000.0,5,2,160,205,25
3,10,98,0,5,increase utilization quality reproductive mate...,4,100000000.0,0.0,100000000.0,100000000.0,0.0,5,8,160,108,12
4,10,99,0,5,project development objective pdo support deve...,4,150000000.0,0.0,150000000.0,150000000.0,0.0,5,8,160,150,17


In [49]:
tfidf = TfidfVectorizer()
df['Project Development Objective '] = tfidf.fit_transform(df['Project Development Objective ']).toarray()

The column Project Development Stage has been vectorized using TfidfVectorizer

In [50]:
a=df.pop('Project Status')
df['Project Status']=a

The dependent variable Project Status has been moved to the end of the dataframe

In [51]:
df.fillna(0, inplace=True)

Any NaN values in the data have been replaced with 0

In [52]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [53]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [54]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()
xgb = XGBClassifier(n_estimators=50, random_state=2)
clfs = {'SVC' : svc,
'KN' : knc,
'NB': mnb,
'DT': dtc,
'LR': lrc,
'RF': rfc,
'AdaBoost': abc,
'BgC': bc,
'ETC': etc,
'GBDT':gbdt,
'GNB':gnb,
'MNB':mnb,
'BNB':bnb,
'XGB':xgb
}

In [55]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    return accuracy,precision
accuracy_scores = []
precision_scores = []
for name,clf in clfs.items():
    current_accuracy,current_precision = train_classifier(clf, x_train,y_train,x_test,y_test)
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Precison':accuracy_scores,'Accuracy':precision_scores}).sort_values('Accuracy',ascending=False)



For  SVC
Accuracy -  0.730099791134834
Precision -  0.730099791134834
For  KN
Accuracy -  0.7535391042005105
Precision -  0.8091988130563799
For  NB
Accuracy -  0.625203063355767
Precision -  0.728849028400598
For  DT
Accuracy -  0.9032258064516129
Precision -  0.976931142957008


c:\Users\ritod\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


For  LR
Accuracy -  0.7904386168484567
Precision -  0.8197889934416881
For  RF
Accuracy -  0.9278254815502437
Precision -  0.952441749122247
For  AdaBoost
Accuracy -  0.9164539336272917
Precision -  0.9476221079691517
For  BgC
Accuracy -  0.9310744952425156
Precision -  0.9561319244316362
For  ETC
Accuracy -  0.9266651195172894
Precision -  0.9523657289002557
For  GBDT
Accuracy -  0.9199350197261545
Precision -  0.9554471544715447
For  GNB
Accuracy -  0.7379902529589232
Precision -  0.7415568862275449
For  MNB
Accuracy -  0.625203063355767
Precision -  0.728849028400598
For  BNB
Accuracy -  0.7338129496402878
Precision -  0.7328208711856511
For  XGB
Accuracy -  0.9282896263634254
Precision -  0.9550850176451716


,Algorithm,Precison,Accuracy
3,DT,0.903226,0.976931
7,BgC,0.931074,0.956132
9,GBDT,0.919935,0.955447
13,XGB,0.928290,0.955085
5,RF,0.927825,0.952442
8,ETC,0.926665,0.952366
6,AdaBoost,0.916454,0.947622
4,LR,0.790439,0.819789
1,KN,0.753539,0.809199
10,GNB,0.737990,0.741557


In [58]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_max_ft_3000':accuracy_scores,'Precision_max_ft_3000':precision_scores}).sort_values('Precision_max_ft_3000',ascending=False)
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_scaling':accuracy_scores,'Precision_scaling':precision_scores}).sort_values('Precision_scaling',ascending=False)
new_df = performance_df.merge(temp_df,on='Algorithm')
new_df_scaled = new_df.merge(temp_df,on='Algorithm')
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_num_chars':accuracy_scores,'Precision_num_chars':precision_scores}).sort_values('Precision_num_chars',ascending=False)
new_df_scaled.merge(temp_df,on='Algorithm')

,Algorithm,Precison,Accuracy,Accuracy_scaling_x,Precision_scaling_x,Accuracy_scaling_y,Precision_scaling_y,Accuracy_num_chars,Precision_num_chars
0,DT,0.903226,0.976931,0.903226,0.976931,0.903226,0.976931,0.903226,0.976931
1,BgC,0.931074,0.956132,0.931074,0.956132,0.931074,0.956132,0.931074,0.956132
2,GBDT,0.919935,0.955447,0.919935,0.955447,0.919935,0.955447,0.919935,0.955447
3,XGB,0.928290,0.955085,0.928290,0.955085,0.928290,0.955085,0.928290,0.955085
4,RF,0.927825,0.952442,0.927825,0.952442,0.927825,0.952442,0.927825,0.952442
5,ETC,0.926665,0.952366,0.926665,0.952366,0.926665,0.952366,0.926665,0.952366
6,AdaBoost,0.916454,0.947622,0.916454,0.947622,0.916454,0.947622,0.916454,0.947622
7,LR,0.790439,0.819789,0.790439,0.819789,0.790439,0.819789,0.790439,0.819789
8,KN,0.753539,0.809199,0.753539,0.809199,0.753539,0.809199,0.753539,0.809199
9,GNB,0.737990,0.741557,0.737990,0.741557,0.737990,0.741557,0.737990,0.741557


In [61]:
# Voting Classifier
svc = SVC(kernel='sigmoid', gamma=1.0,probability=True)
mnb = MultinomialNB()
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(estimators=[('svm', svc), ('nb', mnb), ('et', etc)],voting='soft')
voting.fit(x_train,y_train)

VotingClassifier(estimators=[('svm',
                              SVC(gamma=1.0, kernel='sigmoid',
                                  probability=True)),
                             ('nb', MultinomialNB()),
                             ('et',
                              ExtraTreesClassifier(n_estimators=50,
                                                   random_state=2))],
                 voting='soft')

In [63]:
y_pred = voting.predict(x_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Precision",precision_score(y_test,y_pred))

Accuracy 0.7549315386400557
Precision 0.7641557128412538


In [64]:
from sklearn.ensemble import StackingClassifier
estimators=[('svm', svc), ('nb', mnb), ('et', etc)]
final_estimator=RandomForestClassifier()
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print("Accuracy",accuracy_score(y_test,y_pred))
print("Precision",precision_score(y_test,y_pred))

Accuracy 0.9194708749129729
Precision 0.9489894128970163


## From the above analysis, we can conclude that Decision Tree Classifier is the best model to be used for this problem with an accuracy of 97.7%

This model now can be used to predict whether the project is closed or not. If project status is 1 it means it's closed.